In [6]:
from deepface import DeepFace
import cv2
import os
from scipy.spatial.distance import cosine
import numpy as np

In [7]:
# ======== CONFIG ========
face_db_path = "face_db"
model_name = "VGG-Face"
threshold = 0.4  # Cosine distance threshold
label_font = cv2.FONT_HERSHEY_SIMPLEX
last_label = None
last_distance = 1.0
smoothing_decay = 0.1  # Smoothing factor for flicker reduction


In [8]:
# ======== Load face database ========
print("[INFO] Loading face embeddings from DB...")
database = {}

for person_folder in os.listdir(face_db_path):
    person_path = os.path.join(face_db_path, person_folder)

    if not os.path.isdir(person_path):
        continue  # Skip non-folder files like .pkl

    for img_file in os.listdir(person_path):
        if not img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
            print(f"[SKIPPED] {img_file} is not an image")
            continue

        try:
            img_path = os.path.join(person_path, img_file)
            rep = DeepFace.represent(img_path=img_path, model_name=model_name, enforce_detection=False)[0]["embedding"]

            if person_folder not in database:
                database[person_folder] = []

            database[person_folder].append(rep)
            print(f"[LOADED] {person_folder}/{img_file}")

        except Exception as e:
            print(f"[WARNING] Skipped {img_file} due to error: {e}")

if not database:
    print("[FATAL] No valid face data found in face_db.")
    exit()

[INFO] Loading face embeddings from DB...
[LOADED] Chetan/0.jpg
[LOADED] Chetan/1.jpg
[LOADED] Chetan/10.jpg
[LOADED] Chetan/11.jpg
[LOADED] Chetan/12.jpg
[LOADED] Chetan/13.jpg
[LOADED] Chetan/14.jpg
[LOADED] Chetan/15.jpg
[LOADED] Chetan/16.jpg
[LOADED] Chetan/17.jpg
[LOADED] Chetan/18.jpg
[LOADED] Chetan/19.jpg
[LOADED] Chetan/2.jpg
[LOADED] Chetan/20.jpg
[LOADED] Chetan/21.jpg
[LOADED] Chetan/22.jpg
[LOADED] Chetan/23.jpg
[LOADED] Chetan/24.jpg
[LOADED] Chetan/25.jpg
[LOADED] Chetan/26.jpg
[LOADED] Chetan/3.jpg
[LOADED] Chetan/4.jpg
[LOADED] Chetan/5.jpg
[LOADED] Chetan/6.jpg
[LOADED] Chetan/7.jpg
[LOADED] Chetan/8.jpg
[LOADED] Chetan/9.jpg
[LOADED] Dev/0.jpg
[LOADED] Dev/1.jpg
[LOADED] Dev/10.jpg
[LOADED] Dev/11.jpg
[LOADED] Dev/12.jpg
[LOADED] Dev/13.jpg
[LOADED] Dev/14.jpg
[LOADED] Dev/15.jpg
[LOADED] Dev/16.jpg
[LOADED] Dev/17.jpg
[LOADED] Dev/18.jpg
[LOADED] Dev/19.jpg
[LOADED] Dev/2.jpg
[LOADED] Dev/20.jpg
[LOADED] Dev/21.jpg
[LOADED] Dev/22.jpg
[LOADED] Dev/23.jpg
[LOADED] D

In [9]:
cap = cv2.VideoCapture(0)
print("[INFO] Starting webcam. Press ESC to quit.")

[INFO] Starting webcam. Press ESC to quit.


In [10]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    try:
        face_objs = DeepFace.extract_faces(frame, enforce_detection=False)
        for face in face_objs:
            try:
                x = face['facial_area']['x']
                y = face['facial_area']['y']
                w = face['facial_area']['w']
                h = face['facial_area']['h']
            except Exception as e:
                print(f"[SKIP FACE] Bad bounding box: {e}")
                continue

            cropped_face = face["face"]

            # Represent the live face
            emb = DeepFace.represent(cropped_face, model_name=model_name, enforce_detection=False)[0]["embedding"]

            best_match = "Unknown"
            best_dist = 1.0

            for name, embeddings in database.items():
                for ref_emb in embeddings:
                    dist = cosine(emb, ref_emb)
                    if dist < threshold and dist < best_dist:
                        best_match = f"Detected: {name}"
                        best_dist = dist


            # Flicker smoothing
            if best_dist < last_distance:
                last_label = best_match
                last_distance = best_dist
            else:
                # Decay (if not improving, slowly go back to Unknown)
                last_distance += smoothing_decay
                if last_distance > threshold:
                    last_label = "Detecting..."

            # Draw bounding box and label
            if last_label=="Unknown":
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
            else:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

            cv2.putText(frame, last_label, (x, y-10), label_font, 0.8, (255, 255, 255), 2)

    except Exception as e:
        print(f"[ERROR]: {e}")

    cv2.imshow("Face Recognition - DeepFace", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()